In [1]:
# import prices from csv file
import pandas as pd
prices = pd.read_csv("data/price.csv", index_col=0, header=0, parse_dates=True)

In [3]:
# connect with MongoDB (here using a mock database)
from mongoengine import *
connect(db="test", host='mongomock://localhost', alias='default')

mongomock.MongoClient('localhost', 27017)

In [10]:
from pyutil.mongo.engine.symbol import Symbol, Group

# Here you can go wild and point to an existing MongoDB Server. I use an inmemory-solution here...
g = Group(name="US Stock")
g.save()

<Group: Group object>

In [11]:
for key, data in prices.items():
    symbol = Symbol(name=key, group=g, internal=key)
    # upsert the prices
    symbol.price = data.dropna()
    symbol.save() 

In [13]:
for symbol in Symbol.objects:
    print(symbol.name)
    print(symbol.group.name)
    print(symbol.internal)

A
US Stock
A
B
US Stock
B
C
US Stock
C
D
US Stock
D
E
US Stock
E
F
US Stock
F
G
US Stock
G


In [20]:
s1 = Symbol.objects(name="A").get()
print(s1.name)
print(s1.group.name)
s1.reference["AA"] = 20.0
s1.reference["BB"] = 30.0
s1.save()

s2 = Symbol.objects(name="C").get()
print(s2.name)
print(s2.group.name)
s2.reference["BB"] = 40.0
s2.reference["CC"] = 30.0
s2.save()

Symbol.reference_frame(products=[s1,s2], f=lambda x: x.name)

A
US Stock
C
US Stock


,AA,BB,CC,Sector,Internal
symbol,,,,,
A,20.0,30.0,NaN,US Stock,A
C,NaN,40.0,30.0,US Stock,C


In [22]:
Symbol.pandas_frame(key="price", products=[s1,s2], f=lambda x: x.name)

symbol,A,C
2013-01-01,1673.78,62550.10
2013-01-02,1686.90,62550.10
2013-01-03,1663.95,63312.46
2013-01-04,1655.65,62523.06
2013-01-07,1646.95,61932.54
2013-01-08,1659.25,61127.84
2013-01-09,1657.75,61578.58
2013-01-10,1675.45,61678.31
2013-01-11,1662.80,61497.43
2013-01-14,1667.85,62080.79
